In [3]:
! pip install jupyter-dash dash plotly dash-bootstrap-components pandas statsmodels paho-mqtt
# Documentation - https://dash.plotly.com/dash-core-components

^C


## Simulated 3d Printer for generating readings for dashboard

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Global DataFrame for live simulation
df = pd.DataFrame(columns=[
    'timestamp', 'nozzle_temp_C', 'bed_temp_C',
    'ambient_temp_C', 'ambient_humidity_%',
    'print_progress_%', 'printer_status'
])
MQTT_BROKER = "localhost" 
MQTT_PORT = 1883
MQTT_TOPIC = "sensors/temp&humid"

def generate_new_row():
    if len(df) == 0:
        progress = 0
        status = 'Printing'
    else:
        progress = min(100, df.iloc[-1]['print_progress_%'] + np.random.uniform(0.5, 2.0))
        status = 'Completed' if progress >= 100 else 'Printing'

    return {
        'timestamp': datetime.now(),
        'nozzle_temp_C': np.random.normal(200, 5) if status == 'Printing' else np.random.normal(25, 2),
        'bed_temp_C': np.random.normal(60, 3) if status == 'Printing' else np.random.normal(25, 2),
        'ambient_temp_C': np.random.normal(28, 1),
        'ambient_humidity_%': np.random.normal(45, 5),
        'print_progress_%': progress,
        'printer_status': status
    }

# Preload a few rows
for _ in range(1):
    df = pd.concat([df, pd.DataFrame([generate_new_row()])], ignore_index=True)


C:\Users\yeshw\AppData\Local\Temp\ipykernel_14572\2061472316.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([generate_new_row()])], ignore_index=True)


## DashBoard

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

def tab1_layout(latest):
    gauges = dbc.Row([
        dbc.Col(dcc.Graph(
            figure=go.Figure(go.Indicator(
                mode="gauge+number",
                value=latest['nozzle_temp_C'],
                title={"text": "Nozzle Temp (°C)"},
                gauge={'axis': {'range': [0, 250]}}
            ))
        ), width=6),
        dbc.Col(dcc.Graph(
            figure=go.Figure(go.Indicator(
                mode="gauge+number",
                value=latest['bed_temp_C'],
                title={"text": "Bed Temp (°C)"},
                gauge={'axis': {'range': [0, 120]}}
            ))
        ), width=6)
    ])

    cards = dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardHeader("Ambient Temp (°C)"),
            dbc.CardBody(html.H4(f"{latest['ambient_temp_C']:.2f}"))
        ]), width=3),
        dbc.Col(dbc.Card([
            dbc.CardHeader("Humidity (%)"),
            dbc.CardBody(html.H4(f"{latest['ambient_humidity_%']:.2f}"))
        ]), width=3),
        dbc.Col(dbc.Card([
            dbc.CardHeader("Print Progress (%)"),
            dbc.CardBody(html.H4(f"{latest['print_progress_%']:.1f}"))
        ]), width=3),
        dbc.Col(dbc.Card([
            dbc.CardHeader("Printer Status"),
            dbc.CardBody(html.H4(latest['printer_status']))
        ]), width=3)
    ])

    progress_bar = dbc.Progress(
        value=latest['print_progress_%'],
        label=f"{latest['print_progress_%']:.1f}%",
        style={"height": "30px"},
        color="success"
    )

    return dbc.Container([
        html.H3("Live Overview"),
        cards,
        html.Br(),
        html.H5("Print Progress"),
        progress_bar,

        gauges
    ])

def tab2_layout(local_df):
    fig_line = px.line(local_df, x='timestamp', y=['ambient_temp_C', 'ambient_humidity_%'],
                       labels={'value': 'Measurement', 'timestamp': 'Time', 'variable': 'Parameter'},
                       title="Ambient Temperature & Humidity Over Time")
    return dbc.Container([dcc.Graph(figure=fig_line)])

def tab3_layout(local_df):
    fig_temps = px.line(local_df, x='timestamp', y=['nozzle_temp_C', 'bed_temp_C'],
                        labels={'value': 'Temperature (°C)', 'timestamp': 'Time', 'variable': 'Component'},
                        title="Nozzle & Bed Temperature Over Time")
    return dbc.Container([dcc.Graph(figure=fig_temps)])

def tab4_layout(local_df):
    fig_progress = px.line(local_df, x='timestamp', y='print_progress_%',
                           title="Print Progress Over Time")
    fig_status = px.scatter(local_df, x='timestamp', y=[1]*len(local_df),
                            color='printer_status',
                            title="Printer Status Over Time")
    fig_status.update_yaxes(showticklabels=False)
    return dbc.Container([dcc.Graph(figure=fig_progress), dcc.Graph(figure=fig_status)])

def tab5_layout(local_df):
    stats_df = local_df.describe().reset_index()
    alerts = local_df[(local_df['printer_status'] == 'Printing') & (local_df['nozzle_temp_C'] < 50)]
    alert_text = "No issues detected" if alerts.empty else f"⚠ {len(alerts)} low nozzle temp events detected"
    return dbc.Container([
        html.H5("Statistical Summary"),
        dash_table.DataTable(
            data=stats_df.to_dict('records'),
            columns=[{"name": i, "id": i} for i in stats_df.columns],
            style_table={'overflowX': 'auto'}
        ),
        html.Br(),
        html.H5("Alerts"),
        html.Div(alert_text, style={'color': 'red' if alerts.shape[0] > 0 else 'green'})
    ])

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    html.H1("Industrial 3D Printer Dashboard "),
    dcc.Tabs(id="tabs", value='tab1', children=[
        dcc.Tab(label='Live Overview', value='tab1'),
        dcc.Tab(label='Environmental Conditions', value='tab2'),
        dcc.Tab(label='Printer Performance', value='tab3'),
        dcc.Tab(label='Process Monitoring', value='tab4'),
        dcc.Tab(label='Diagnostics', value='tab5'),
    ]),
    html.Div(id='tabs-content'),
    dcc.Interval(id='interval-component', interval=5000, n_intervals=0)  # every 5 sec
], fluid=True)

@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value'),
    Input('interval-component', 'n_intervals')
)
def render_content(tab, n):
    global df
    beforelatest = df.iloc[-1]
    if beforelatest['printer_status']=='Printing':
        df = pd.concat([df, pd.DataFrame([generate_new_row()])], ignore_index=True)
    latest = df.iloc[-1]
    if tab == 'tab1':
        return tab1_layout(latest)
    elif tab == 'tab2':
        return tab2_layout(df)
    elif tab == 'tab3':
        return tab3_layout(df)
    elif tab == 'tab4':
        return tab4_layout(df)
    elif tab == 'tab5':
        return tab5_layout(df)

app.run()
